# Machine Learning With Spark ML
In this notebook, the following steps have been followed:
1. Get the data.
2. Discover the data to gain insights.
3. Prepare the data for Machine Learning algorithms.
4. Select a model and train it.
5. Fine-tune the model.
6. Present the solution.

As a dataset, we use the California Housing Prices dataset from the StatLib repository. This dataset was based on data from the 1990 California census. The dataset has the following columns
1. `longitude`: a measure of how far west a house is (a higher value is farther west)
2. `latitude`: a measure of how far north a house is (a higher value is farther north)
3. `housing_,median_age`: median age of a house within a block (a lower number is a newer building)
4. `total_rooms`: total number of rooms within a block
5. `total_bedrooms`: total number of bedrooms within a block
6. `population`: total number of people residing within a block
7. `households`: total number of households, a group of people residing within a home unit, for a block
8. `median_income`: median income for households within a block of houses
9. `median_house_value`: median house value for households within a block
10. `ocean_proximity`: location of the house w.r.t ocean/sea

---
# 1. Get the data
Let's start the lab by loading the dataset. The can find the dataset at `data/housing.csv`. To infer column types automatically, when you are reading the file, you need to set `inferSchema` to true. Moreover enable the `header` option to read the columns' name from the file.

In [1]:
## Installing PySpark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 60.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=48937851d74228ef2dcc37b9b40b40f1aa17bd1b673869de39d615f15bf7f30f
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
## Starting a Spark Session and Naming it
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Lab1').getOrCreate()
spark

In [3]:
## Reading the Data. Make sure the files are uploaded in the session [on the left under the folder icon]
housing = spark.read.csv('housing.csv', header = True, inferSchema = True)

---
# 2. Discover the data to gain insights
Now it is time to take a look at the data. In this step we are going to take a look at the data a few different ways:
* See the schema and dimension of the dataset
* Look at the data itself
* Statistical summary of the attributes
* Breakdown of the data by the categorical attribute variable
* Find the correlation among different attributes
* Make new attributes by combining existing attributes

## 2.1. Schema and dimension
Print the schema of the dataset

In [4]:
## TODO: Printing the Schema
housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Print the number of records in the dataset.

In [5]:
## TODO: Printing the number of records in the dataset
housing.count()

20640

## 2.2. Look at the data
Print the first five records of the dataset.

In [6]:
## TODO: Printing the first five records 
housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [7]:
## TODO: Printing the first five records 
housing.head(5)

[Row(longitude=-122.23, latitude=37.88, housing_median_age=41.0, total_rooms=880.0, total_bedrooms=129.0, population=322.0, households=126.0, median_income=8.3252, median_house_value=452600.0, ocean_proximity='NEAR BAY'),
 Row(longitude=-122.22, latitude=37.86, housing_median_age=21.0, total_rooms=7099.0, total_bedrooms=1106.0, population=2401.0, households=1138.0, median_income=8.3014, median_house_value=358500.0, ocean_proximity='NEAR BAY'),
 Row(longitude=-122.24, latitude=37.85, housing_median_age=52.0, total_rooms=1467.0, total_bedrooms=190.0, population=496.0, households=177.0, median_income=7.2574, median_house_value=352100.0, ocean_proximity='NEAR BAY'),
 Row(longitude=-122.25, latitude=37.85, housing_median_age=52.0, total_rooms=1274.0, total_bedrooms=235.0, population=558.0, households=219.0, median_income=5.6431, median_house_value=341300.0, ocean_proximity='NEAR BAY'),
 Row(longitude=-122.25, latitude=37.85, housing_median_age=52.0, total_rooms=1627.0, total_bedrooms=280.0,

Print the number of records with population more than 10000.

In [8]:
## TODO: Printing number of records with population more than 10000
housing.filter(housing['population']>10000).count()

23

## 2.3. Statistical summary
Print a summary of the table statistics for the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. You can use the `describe` command.

In [9]:
## Statistical summary
housing.select(['housing_median_age', 'total_rooms', 'median_house_value', 'population']).describe().show()

+-------+------------------+------------------+------------------+------------------+
|summary|housing_median_age|       total_rooms|median_house_value|        population|
+-------+------------------+------------------+------------------+------------------+
|  count|             20640|             20640|             20640|             20640|
|   mean|28.639486434108527|2635.7630813953488|206855.81690891474|1425.4767441860465|
| stddev| 12.58555761211163|2181.6152515827944|115395.61587441359|  1132.46212176534|
|    min|               1.0|               2.0|           14999.0|               3.0|
|    max|              52.0|           39320.0|          500001.0|           35682.0|
+-------+------------------+------------------+------------------+------------------+



Print the maximum age (`housing_median_age`), the minimum number of rooms (`total_rooms`), and the average of house values (`median_house_value`).

In [10]:
## TODO: 
#  1. Print the max age 
#  2. Print the minumum number of rooms
#  3. Print the average of house values
housing.agg({'housing_median_age':'max'}).show()
housing.agg({'total_rooms':'min'}).show()
housing.agg({'median_house_value':'mean'}).show()

+-----------------------+
|max(housing_median_age)|
+-----------------------+
|                   52.0|
+-----------------------+

+----------------+
|min(total_rooms)|
+----------------+
|             2.0|
+----------------+

+-----------------------+
|avg(median_house_value)|
+-----------------------+
|     206855.81690891474|
+-----------------------+



## 2.4. Breakdown the data by categorical data
Print the number of houses in different areas (`ocean_proximity`), and sort them in descending order.

In [11]:
## TODO: Print the number of houses in different areas and sort them in descending order
housing.groupBy('ocean_proximity').count().sort('count', ascending = False).show()

+---------------+-----+
|ocean_proximity|count|
+---------------+-----+
|      <1H OCEAN| 9136|
|         INLAND| 6551|
|     NEAR OCEAN| 2658|
|       NEAR BAY| 2290|
|         ISLAND|    5|
+---------------+-----+



Print the average value of the houses (`median_house_value`) in different areas (`ocean_proximity`), and call the new column `avg_value` when print it.

In [12]:
## TODO: 
housing.groupBy('ocean_proximity').agg({'median_house_value':'mean'}).withColumnRenamed('avg(median_house_value)','avg_value').show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



Rewrite the above question in SQL.

In [13]:
## Reqriting as SQL query
housing.createOrReplaceTempView("df")
spark.sql("SELECT ocean_proximity, AVG(median_house_value) as avg_value FROM df GROUP BY ocean_proximity").show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



## 2.5. Correlation among attributes
Print the correlation among the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. To do so, first you need to put these attributes into one vector. Then, compute the standard correlation coefficient (Pearson) between every pair of attributes in this new vector. To make a vector of these attributes, you can use the `VectorAssembler` Transformer.

In [14]:
## TODO:
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler(inputCols = ["housing_median_age", "total_rooms", "median_house_value", "population"], outputCol = "Combined Attributes")
housingAttrs = va.transform(housing)
housingAttrs.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity| Combined Attributes|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|[41.0,880.0,45260...|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|[21.0,7099.0,3585...|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|[52.0,1467.0,3521...|
|  -122.25|   37.85|              52.0|     12

In [15]:
from pyspark.ml.linalg import DenseMatrix, Vectors
from pyspark.ml.stat import Correlation

pearsonCorr = Correlation.corr(housingAttrs, 'Combined Attributes', 'pearson').collect()[0][0]
print("The standard correlation coefficient: \n {}".format(pearsonCorr))

The standard correlation coefficient: 
 DenseMatrix([[ 1.        , -0.3612622 ,  0.10562341, -0.29624424],
             [-0.3612622 ,  1.        ,  0.13415311,  0.85712597],
             [ 0.10562341,  0.13415311,  1.        , -0.02464968],
             [-0.29624424,  0.85712597, -0.02464968,  1.        ]])


## 2.6. Combine and make new attributes
Now, let's try out various attribute combinations. In the given dataset, the total number of rooms in a block is not very useful, if we don't know how many households there are. What we really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful, and we want to compare it to the number of rooms. And the population per household seems like also an interesting attribute combination to look at. To do so, add the three new columns to the dataset as below. We will call the new dataset the `housingExtra`.
```
rooms_per_household = total_rooms / households
bedrooms_per_room = total_bedrooms / total_rooms
population_per_household = population / households
```

In [16]:
## TODO:
housingCol1 = housing.withColumn('rooms_per_household', housing['total_rooms']/housing['households'])
housingCol2 = housingCol1.withColumn('bedrooms_per_room', housing['total_bedrooms']/housing['total_rooms'])
housingExtra = housingCol2.withColumn('population_per_household', housing['population']/housing['households'])
housingExtra.select("rooms_per_household", "bedrooms_per_room", "population_per_household").show(5)

+-------------------+-------------------+------------------------+
|rooms_per_household|  bedrooms_per_room|population_per_household|
+-------------------+-------------------+------------------------+
|  6.984126984126984|0.14659090909090908|      2.5555555555555554|
|  6.238137082601054|0.15579659106916466|       2.109841827768014|
|  8.288135593220339|0.12951601908657123|      2.8022598870056497|
| 5.8173515981735155|0.18445839874411302|       2.547945205479452|
|  6.281853281853282| 0.1720958819913952|      2.1814671814671813|
+-------------------+-------------------+------------------------+
only showing top 5 rows



---
## 3. Prepare the data for Machine Learning algorithms
Before going through the Machine Learning steps, let's first rename the label column from `median_house_value` to `label`.

In [17]:
## TODO
renamedHousing = housingExtra.withColumnRenamed('median_house_value','label')

Now, we want to separate the numerical attributes from the categorical attribute (`ocean_proximity`) and keep their column names in two different lists. Moreover, sice we don't want to apply the same transformations to the predictors (features) and the label, we should remove the label attribute from the list of predictors. 

In [18]:
##
colLabel = "label"

colCat   = "ocean_proximity"
colNum   = renamedHousing.drop(colLabel, colCat)

## 3.1. Prepare continuous attributes
### Data cleaning
Most Machine Learning algorithms cannot work with missing features, so we should take care of them. As a first step, let's find the columns with missing values in the numerical attributes. To do so, we can print the number of missing values of each continues attributes, listed in `colNum`.

In [19]:
from pyspark.sql.functions import col,sum
renamedHousing.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in renamedHousing.columns)).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+-----+---------------+-------------------+-----------------+------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|label|ocean_proximity|rooms_per_household|bedrooms_per_room|population_per_household|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+-----+---------------+-------------------+-----------------+------------------------+
|        0|       0|                 0|          0|           207|         0|         0|            0|    0|              0|                  0|              207|                       0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+-----+---------------+-------------------+-----------------+------------------------+



As we observerd above, the `total_bedrooms` and `bedrooms_per_room` attributes have some missing values. One way to take care of missing values is to use the `Imputer` Transformer, which completes missing values in a dataset, either using the mean or the median of the columns in which the missing values are located. To use it, you need to create an `Imputer` instance, specifying that you want to replace each attribute's missing values with the "median" of that attribute.

In [20]:
## Imputer
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['total_bedrooms', 'bedrooms_per_room'], 
    outputCols=['total_bedrooms', 'bedrooms_per_room']
    ).setStrategy("median")
imputedHousing = imputer.fit(renamedHousing).transform(renamedHousing)

#imputedHousing.show(5)
imputedHousing.select("total_bedrooms", "bedrooms_per_room").show(5)

+--------------+-------------------+
|total_bedrooms|  bedrooms_per_room|
+--------------+-------------------+
|         129.0|0.14659090909090908|
|        1106.0|0.15579659106916466|
|         190.0|0.12951601908657123|
|         235.0|0.18445839874411302|
|         280.0| 0.1720958819913952|
+--------------+-------------------+
only showing top 5 rows



### Scaling
One of the most important transformations you need to apply to the data is feature scaling. With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15. Note that scaling the label attribues is generally not required.

One way to get all attributes to have the same scale is to use standardization. In standardization, for each value, first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the variance so that the resulting distribution has unit variance. To do this, we can use the `StandardScaler` Estimator. To use `StandardScaler`, again we need to convert all the numerical attributes into a big vectore of features using `VectorAssembler`, and then call `StandardScaler` on that vactor.

In [21]:
##
from pyspark.ml.feature import StandardScaler

va = VectorAssembler(inputCols=colNum.columns, outputCol='Combined Features')
featuredHousing = va.transform(imputedHousing)

scaler = StandardScaler(inputCol='Combined Features', outputCol='Combined Features Scaled')
scaledHousing = scaler.fit(featuredHousing).transform(featuredHousing)

scaledHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|   Combined Features|Combined Features Scaled|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+------------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|[-122.23,37.88,41...|    [-61.007269596069...|
|  -122.22|   37.86|              21.0|     

## 3.2. Prepare categorical attributes
After imputing and scaling the continuse attributes, we should take care of the categorical attributes. Let's first print the number of distict values of the categirical attribute `ocean_proximity`.

In [22]:
##
renamedHousing.select('ocean_proximity').distinct().show()

+---------------+
|ocean_proximity|
+---------------+
|         ISLAND|
|     NEAR OCEAN|
|       NEAR BAY|
|      <1H OCEAN|
|         INLAND|
+---------------+



### String indexer
Most Machine Learning algorithms prefer to work with numbers. So let's convert the categorical attribute `ocean_proximity` to numbers. To do so, we can use the `StringIndexer` that encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.

In [23]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol = "ocean_proximity", outputCol = "ocean_proximity_indexed")
idxHousing = indexer.fit(renamedHousing).transform(renamedHousing)
idxHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|ocean_proximity_indexed|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|                    3.0|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|358500.0|       NEAR BAY|  6.2

Now we can use this numerical data in any Machine Learning algorithm. You can look at the mapping that this encoder has learned using the `labels` method: "<1H OCEAN" is mapped to 0, "INLAND" is mapped to 1, etc.

In [24]:
indexer.fit(renamedHousing).labels

['<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'NEAR BAY', 'ISLAND']

### One-hot encoding
Now, convert the label indices built in the last step into one-hot vectors. To do this, you can take advantage of the `OneHotEncoderEstimator` Estimator.

In [25]:
##

from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCol="ocean_proximity_indexed", outputCol="ocean_proximity_encoded")
ohHousing = encoder.fit(idxHousing).transform(idxHousing)

ohHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+-----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|ocean_proximity_indexed|ocean_proximity_encoded|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+-----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|                    3.0|          (4,[3],[1.0])|
|  -122.22|   37.86|              21

---
# 4. Pipeline
As you can see, there are many data transformation steps that need to be executed in the right order. For example, you called the `Imputer`, `VectorAssembler`, and `StandardScaler` from left to right. However, we can use the `Pipeline` class to define a sequence of Transformers/Estimators, and run them in order. A `Pipeline` is an `Estimator`, thus, after a Pipeline's `fit()` method runs, it produces a `PipelineModel`, which is a `Transformer`.

Now, let's create a pipeline called `numPipeline` to call the numerical transformers you built above (`imputer`, `va`, and `scaler`) in the right order from left to right, as well as a pipeline called `catPipeline` to call the categorical transformers (`indexer` and `encoder`). Then, put these two pipelines `numPipeline` and `catPipeline` into one pipeline.

In [26]:
##
from pyspark.ml import Pipeline

numPipeline = Pipeline(stages = [imputer, va, scaler])
catPipeline = Pipeline(stages = [indexer, encoder])
pipeline    = Pipeline(stages = [numPipeline, catPipeline])
newHousing  = pipeline.fit(renamedHousing).transform(renamedHousing)

newHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+------------------------+-----------------------+-----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|   Combined Features|Combined Features Scaled|ocean_proximity_indexed|ocean_proximity_encoded|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+------------------------+-----------------------+-----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.98412698

Now, use `VectorAssembler` to put all attributes of the final dataset `newHousing` into a big vector, and call the new column `features`.

In [27]:
va2 = VectorAssembler(inputCols=['Combined Features Scaled','ocean_proximity_encoded'], outputCol = 'features')
dataset = va2.transform(newHousing)

dataset.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+------------------------+-----------------------+-----------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|   Combined Features|Combined Features Scaled|ocean_proximity_indexed|ocean_proximity_encoded|            features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+------------------------+-----------------------+-----------------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0

---
# 5. Make a model
Here we going to make four different regression models:
* Linear regression model
* Decision tree regression
* Random forest regression
* Gradient-booster forest regression

But, before giving the data to train a Machine Learning model, let's first split the data into training dataset (`trainSet`) with 80% of the whole data, and test dataset (`testSet`) with 20% of it.

In [28]:
## Extracting relevant data - features and label
finalized_data = dataset.select('features','label')
finalized_data.show()

+--------------------+--------+
|            features|   label|
+--------------------+--------+
|[-61.007269596069...|452600.0|
|[-61.002278409814...|358500.0|
|[-61.012260782324...|352100.0|
|[-61.017251968579...|341300.0|
|[-61.017251968579...|342200.0|
|[-61.017251968579...|269700.0|
|[-61.017251968579...|299200.0|
|[-61.017251968579...|241400.0|
|[-61.022243154834...|226700.0|
|[-61.017251968579...|261100.0|
|[-61.022243154834...|281500.0|
|[-61.022243154834...|241800.0|
|[-61.022243154834...|213500.0|
|[-61.022243154834...|191300.0|
|[-61.022243154834...|159200.0|
|[-61.022243154834...|140000.0|
|[-61.027234341089...|152500.0|
|[-61.027234341089...|155500.0|
|[-61.022243154834...|158700.0|
|[-61.027234341089...|162900.0|
+--------------------+--------+
only showing top 20 rows



In [29]:
## Splitting the data 80-20
trainSet,testSet=finalized_data.randomSplit([0.80,0.20])

## 5.1. Linear regression model
Now, train a Linear Regression model using the `LinearRegression` class. Then, print the coefficients and intercept of the model, as well as the summary of the model over the training set by calling the `summary` method.

In [30]:
##
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

lr = LinearRegression(featuresCol = 'features', labelCol = 'label')
lrModel = lr.fit(trainSet)
trainingSummary = lrModel.summary
print("Coefficients: {}, Intercept: {}".format(lrModel.coefficients, lrModel.intercept))
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))

Coefficients: [-55160.8073859172,-55227.11866477375,13426.95722668069,6849.135586238802,1356.9126309644705,-46997.60390299709,43983.75037091673,77549.51837766099,7022.110667914757,15852.495055309202,2633.3573870669543,-166823.2440543088,-202875.86694466273,-163888.21160193806,-171652.9394087955], Intercept: -2256895.030124878
RMSE: 68246.80916245039


Now, use `RegressionEvaluator` to measure the root-mean-square-erroe (RMSE) of the model on the test dataset.

In [31]:
##Prediction on test data
pred_results = lrModel.transform(testSet)
pred_results.select("prediction", "label", "features").show(5)

##RMSE of Test Set
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(pred_results)
print("Root Mean Squared Error (RMSE) on test data: {}".format(rmse))

+------------------+-------+--------------------+
|        prediction|  label|            features|
+------------------+-------+--------------------+
|184101.07485990692|76100.0|[-62.015489219599...|
| 208541.4914438515|50800.0|[-62.005506847089...|
|117274.93193060905|73200.0|[-62.005506847089...|
|162023.19410721445|70200.0|[-61.980550915813...|
| 134113.8653363865|70500.0|[-61.980550915813...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data: 66574.81092352736


## 5.2. Decision tree regression
Repeat what you have done on Regression Model to build a Decision Tree model. Use the `DecisionTreeRegressor` to make a model and then measure its RMSE on the test dataset.

In [32]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator

dt = DecisionTreeRegressor(featuresCol = 'features', labelCol = 'label')
dtModel = dt.fit(trainSet)

##Prediction on test data
pred_results = dtModel.transform(testSet)
pred_results.select("prediction", "label", "features").show(5)

##RMSE of Test Set
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(pred_results)
print("Root Mean Squared Error (RMSE) on test data: {}".format(rmse))

+------------------+-------+--------------------+
|        prediction|  label|            features|
+------------------+-------+--------------------+
| 133129.1208791209|76100.0|[-62.015489219599...|
|171965.91628959277|50800.0|[-62.005506847089...|
|171965.91628959277|73200.0|[-62.005506847089...|
|171965.91628959277|70200.0|[-61.980550915813...|
| 133129.1208791209|70500.0|[-61.980550915813...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data: 67022.06531952161


## 5.3. Random forest regression
Let's try the test error on a Random Forest Model. Youcan use the `RandomForestRegressor` to make a Random Forest model.

In [33]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

rf = RandomForestRegressor(featuresCol = 'features', labelCol = 'label')
rfModel = rf.fit(trainSet)

##Prediction on test data
pred_results = rfModel.transform(testSet)
pred_results.select("prediction", "label", "features").show(5)

##RMSE of Test Set
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(pred_results)
print("Root Mean Squared Error (RMSE) on test data: {}".format(rmse))

+------------------+-------+--------------------+
|        prediction|  label|            features|
+------------------+-------+--------------------+
| 198352.7991661064|76100.0|[-62.015489219599...|
|243204.96480685295|50800.0|[-62.005506847089...|
|193557.55995581218|73200.0|[-62.005506847089...|
|163360.21622017297|70200.0|[-61.980550915813...|
|155372.63181582183|70500.0|[-61.980550915813...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data: 64110.32162233181


## 5.4. Gradient-boosted tree regression
Fianlly, we want to build a Gradient-boosted Tree Regression model and test the RMSE of the test data. Use the `GBTRegressor` to build the model.

In [34]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

gb = GBTRegressor(featuresCol = 'features', labelCol = 'label')
gbModel = gb.fit(trainSet)

##Prediction on test data
pred_results = gbModel.transform(testSet)
pred_results.select("prediction", "label", "features").show(5)

##RMSE of Test Set
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(pred_results)
print("Root Mean Squared Error (RMSE) on test data: {}".format(rmse))

+------------------+-------+--------------------+
|        prediction|  label|            features|
+------------------+-------+--------------------+
|  82556.4076570584|76100.0|[-62.015489219599...|
|131542.35526089705|50800.0|[-62.005506847089...|
|115416.61283072784|73200.0|[-62.005506847089...|
| 97986.01627620768|70200.0|[-61.980550915813...|
| 92073.96956621467|70500.0|[-61.980550915813...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data: 54310.89025697337


---
# 6. Hyperparameter tuning
An important task in Machie Learning is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as LinearRegression, or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately. MLlib supports model selection tools, such as `CrossValidator`. These tools require the following items:
* Estimator: algorithm or Pipeline to tune (`setEstimator`)
* Set of ParamMaps: parameters to choose from, sometimes called a "parameter grid" to search over (`setEstimatorParamMaps`)
* Evaluator: metric to measure how well a fitted Model does on held-out test data (`setEvaluator`)

`CrossValidator` begins by splitting the dataset into a set of folds, which are used as separate training and test datasets. For example with `k=3` folds, `CrossValidator` will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. To evaluate a particular `ParamMap`, `CrossValidator` computes the average evaluation metric for the 3 Models produced by fitting the Estimator on the 3 different (training, test) dataset pairs. After identifying the best `ParamMap`, `CrossValidator` finally re-fits the Estimator using the best ParamMap and the entire dataset.

Below, use the `CrossValidator` to select the best Random Forest model. To do so, you need to define a grid of parameters. Let's say we want to do the search among the different number of trees (1, 5, and 10), and different tree depth (5, 10, and 15).

In [35]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
pipeline = Pipeline(stages=[rf])
paramGrid = ParamGridBuilder().baseOn({rf.labelCol: 'label', rf.predictionCol: 'prediction'}).addGrid(rf.numTrees,[1,5,10]).addGrid(rf.maxDepth,[5,10,15]).build()

In [36]:
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, evaluator=RegressionEvaluator())
cvModel = cv.fit(trainSet) 

In [37]:
predictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

rmse = evaluator.evaluate(pred_results)
print("Root Mean Squared Error (RMSE) on test data: {}".format(rmse))

+------------------+-------+--------------------+
|        prediction|  label|            features|
+------------------+-------+--------------------+
|124155.71428571428|76100.0|[-62.015489219599...|
|145176.73913043478|50800.0|[-62.005506847089...|
|          116925.0|73200.0|[-62.005506847089...|
|           95218.4|70200.0|[-61.980550915813...|
| 85806.52380952382|70500.0|[-61.980550915813...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data: 54310.89025697337
